In [4]:
# 将处理好的acitivity与attention数据合并到一张表上
# 这样方便绘制双线图

import pymysql

# 连接到 MySQL 数据库 
conn = pymysql.connect(
    host='localhost',
    user='root',
    password='cptbtptp789',
    database='project_data',
    charset='utf8mb4'
)

cursor = conn.cursor()

# 删除原表 combined_metrics（如果存在）
drop_table_query = "DROP TABLE IF EXISTS combined_metrics"
try:
    cursor.execute(drop_table_query)
    print("原表 combined_metrics 已删除（如果存在）")
except pymysql.Error as e:
    print(f"删除原表 combined_metrics 时出错: {e}")

# 创建新表 combined_metrics
create_table_query = """
CREATE TABLE IF NOT EXISTS combined_metrics (
    id INT AUTO_INCREMENT PRIMARY KEY,
    project_name VARCHAR(255) NOT NULL,
    month VARCHAR(7) NOT NULL,
    activity_value DECIMAL(10, 2),
    attention_value DECIMAL(10, 2)
)
"""
try:
    cursor.execute(create_table_query)
    print("表 combined_metrics 创建成功")
except pymysql.Error as e:
    print(f"创建表 combined_metrics 时出错: {e}")

# 插入数据
insert_query = """
INSERT INTO combined_metrics (project_name, month, activity_value, attention_value)
SELECT
    a.project_name,
    a.month,
    a.value AS activity_value,
    b.value AS attention_value
FROM
    activity_detailed_metrics a
LEFT JOIN
    attention_detailed_metrics b
ON
    a.project_name = b.project_name AND a.month = b.month
UNION
SELECT
    b.project_name,
    b.month,
    a.value AS activity_value,
    b.value AS attention_value
FROM
    attention_detailed_metrics b
LEFT JOIN
    activity_detailed_metrics a
ON
    b.project_name = a.project_name AND b.month = a.month
WHERE
    a.value IS NULL
"""
try:
    cursor.execute(insert_query)
    conn.commit()
    print("数据插入成功")
except pymysql.Error as e:
    print(f"插入数据时出错: {e}")

# 关闭连接
cursor.close()
conn.close()

原表 combined_metrics 已删除（如果存在）
表 combined_metrics 创建成功
数据插入成功
